In [135]:
import pandas as pd
import numpy as np
import operator

Questions for Christy:
1.) EAB column
2.) Disease Def column: differences between values
3.) What makes a street diverse?

In [114]:
df = pd.read_csv("Trees_Public.csv")
df.head(5)

,FID,ID,UNIQUEID,ADDRESS,SUFFIX,STREET,ONSTR,FROMSTR,TOSTR,SIDE,...,GUARDSZ,EAB,INV_DATE,INV_TIME,INSPECT_DT,INSPECT_TM,NOTES,ACTIVE,COMMONNAME,GENUS
0,0,59084.0,JS 20150203131235,601,,10TH ST,10TH ST,BASELINE RD,CUL-DE-SAC,Front,...,Unassigned,Unassigned,2015-02-03 00:00:00,,2015-07-10 00:00:00,11:40:29,,1.0,Blue Spruce,Picea
1,1,59085.0,JS 20150203131431,601,,10TH ST,10TH ST,BASELINE RD,CUL-DE-SAC,Front,...,Unassigned,Unassigned,2015-02-03 00:00:00,,2015-07-10 00:00:00,11:40:29,,1.0,Blue Spruce,Picea
2,2,86480.0,DROID 201607141607903829845,601,,10TH ST,10TH ST,BASELINE RD,CUL-DE-SAC,Front,...,Unassigned,Unassigned,2016-07-14 00:00:00,,2017-03-09 00:00:00,12:46:32,TC np.,1.0,Siberian Elm,Ulmus
3,3,59087.0,JS 20150203131448,601,,10TH ST,10TH ST,BASELINE RD,CUL-DE-SAC,Front,...,Unassigned,Unassigned,2015-02-03 00:00:00,,2015-07-10 00:00:00,11:40:30,,1.0,Norway Maple,Acer
4,4,59088.0,JS 20150203131959,690,,10TH ST,10TH ST,CUL-DE-SAC,BASELINE RD,Front,...,Unassigned,Unassigned,2015-02-03 00:00:00,,2015-07-10 00:00:00,11:40:30,,1.0,Norway Maple,Acer


In [5]:
df.columns

Index(['FID', 'ID', 'UNIQUEID', 'ADDRESS', 'SUFFIX', 'STREET', 'ONSTR',
       'FROMSTR', 'TOSTR', 'SIDE', 'SITE', 'SPP', 'DBH', 'CULTIVAR',
       'CONDITION', 'INSPECT', 'MT', 'MT2', 'TTYPE', 'MEMTREE', 'DISEASEDEF',
       'STRUTDEFEC', 'LOCTYPE', 'SITECAT', 'HOOD', 'LOCATION', 'JURISDIC',
       'TSIP', 'GRATESZ', 'GUARDSZ', 'EAB', 'INV_DATE', 'INV_TIME',
       'INSPECT_DT', 'INSPECT_TM', 'NOTES', 'ACTIVE', 'COMMONNAME', 'GENUS'],
      dtype='object')

In [187]:
dic(df['DISEASEDEF'])

[(nan, 46756),
 ('signs of stress', 678),
 ('needle necrosis', 548),
 ('other - see notes', 492),
 ('cankers', 486),
 ('crown dieback', 475),
 ('sunscald', 317),
 ('borers', 227),
 ('scale insects', 202),
 ('hawthorn mealybug', 102),
 ('thin canopy', 80),
 ('emerald ash borer', 74),
 ('lilac-ash borer', 50),
 ('chlorosis', 40),
 ('sapsucker damage', 36),
 ('anthracnose', 24),
 ('foliage disorder', 19),
 ('woodpecker damage', 17),
 ('fire blight', 14),
 ('hypoxylon canker', 11),
 ('carpenter ants', 6),
 ('armillaria', 5),
 ('drippy blight / Kermes scale', 3),
 ('frass', 3),
 ('pine wilt nematode', 2),
 ('European elm scale', 1),
 ('thousand cankers disease', 1)]

In [156]:
def dic(c):
    d = {}
    for i in c:
        try:
            d[i]+=1
        except:
            d[i]=1
    import operator
    d = sorted(d.items(), key=operator.itemgetter(1),reverse = True)
    return d
def dic2(c):
    a = {}
    for i in c:
        try:
            a[i]+=1
        except:
            a[i]=1
    import operator
    a = sorted(a.items(), key=operator.itemgetter(1),reverse = True)
    t=0
    for i in a:
        t+=i[1]
    d = []
    for i in a:
        d.append([i[0],float(i[1])/t])
    return d
def dic3(c):
    d={}
    for i in c:
        try:
            d[df[df['STREET']==i[0]]['HOOD'].iloc[0]]+=i[1]
        except:
            d[df[df['STREET']==i[0]]['HOOD'].iloc[0]]=i[1]
    for i in df['HOOD']:
        try:
            d[i]
        except:
            if i == i:
                d[i]=0
    d = sorted(d.items(), key=operator.itemgetter(1),reverse = True)
    return d

In [252]:
ash = df[df['COMMONNAME']=='Green Ash'].append(df[df['COMMONNAME']=='White Ash']) #Subset of tree dataset that only has ash trees
mt_dic = dic(ash['MT'])  #dictionary of sorted values of Maintenance columns

strs_tree_counts={}
for i in df['STREET'].unique():
    e={}
    for j in df[df['STREET']==i]['COMMONNAME']:
        #e['Ash']=0
        if j !="White Ash" and j!="Green Ash":
            try:
                e[j]+=1
            except:
                e[j]=1
        else:
            try:
                e['Ash']+=1
            except:
                e['Ash']=1
    strs_tree_counts[i]=[np.sum([values for key,values in e.items()]),e]
for key,value in strs_tree_counts.items():
    for a,b in value[1].items():
        strs_tree_counts[key][1][a] = [b,float(b)/value[0]]

bad_ash = ash[ash['MT']!='No Specific Maintenance Need']
good_ash = ash[ash['MT']=='No Specific Maintenance Need']

boulder_trees = dic(df['COMMONNAME'])

bad_ash_strs =  [i for i in bad_ash['STREET'].unique()]
bad_ash_strs_counts = sorted([[i,len(bad_ash[bad_ash['STREET'] ==i])] for i in bad_ash_strs], key = lambda num:num[1], reverse = True)
good_ash_strs = [i for i in good_ash['STREET'].unique()]
only_good_ash_strs = [i for i in good_ash['STREET'].unique() if i not in bad_ash_strs]
only_bad_ash_strs = [i for i in bad_ash['STREET'].unique() if i not in good_ash_strs]

bad_strs_percent = len(bad_ash_strs)/len(strs)
good_strs_percent = len(good_ash_strs)/len(strs)

bad_ash_tofro_strs = bad_ash['TOSTR'].append(bad_ash['FROMSTR']).unique()

strs = df['STREET'].append(df['TOSTR']).append(df['FROMSTR']).append(df['ONSTR']).unique()

at_risk_strs = [i for i in only_good_ash_strs if i in bad_ash_tofro_strs]

at_risk_strs_counts = []
for i in at_risk_strs:
    at_risk_strs_counts.append([i,len(ash[ash['STREET']==i])])
    
at_risk_strs_sorted = sorted(at_risk_strs_counts, key=lambda num:num[1], reverse = True)

at_risk_strs_d = {}
for i in at_risk_strs_sorted:
    at_risk_strs_d[i[0]]=i[1]
    
at_risk_strs_ash_dic = {}
for i in at_risk_strs_sorted:
    at_risk_strs_ash_dic[i[0]]=strs_tree_counts[i[0]][1]['Ash']
    
at_risk_hoods=dic3(at_risk_strs_sorted)
bad_ash_hoods_counts = dic3(bad_ash_strs_counts)


strs_trees_percent_dic = {}
for i in df['COMMONNAME'].unique():
    strs_trees_percent_dic[i]=np.mean([value[1][i][1] for key,value in strs_tree_counts.items() if i in value[1]])
t=[value[1]['Ash'][1] for key,value in strs_tree_counts.items() if 'Ash'in value[1]]
strs_trees_percent_dic['Ash']=np.mean(t)
strs_trees_percent_dic_sorted = sorted(strs_trees_percent_dic.items(), key=operator.itemgetter(1),reverse = True)

strs_trees_percent_dic_2 = {}
for i in df['COMMONNAME'].unique():
    t = []
    for key,value in strs_tree_counts.items():
        if i in value[1]:
            t.append(np.mean(value[1][i][1]))
        else:
            t.append(0)
    strs_trees_percent_dic_2[i]=np.mean(t)
t=[]
for key,value in strs_tree_counts.items():
    if 'Ash' in value[1]:
        t.append(value[1]['Ash'][1])
    else:
        t.append(0)
strs_trees_percent_dic_2['Ash']=np.mean(t)
strs_trees_percent_dic_sorted_2 = sorted(strs_trees_percent_dic_2.items(), key=operator.itemgetter(1),reverse = True)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ash: filtered Data frame that only includes ash trees
mt_dic: sorted list of maintenance values for ash trees
boulder_trees: quantities of each tree in boulder

bad_ash: data frame
bad_ash_strs
only_bad_ash_strs
bad_ash_strs_counts
bad_strs_percent
bad_ash_tofro_strs
bad_ash_hoods_counts

good_ash
good_ash_strs
only_good_ash_strs
good_strs_percent

at_risk_strs
at_risk_strs_d
at_risk_strs_sorted
at_risk_strs_ash_dic
at_risk_hoods

strs_trees_percent_dic
strs_trees_percent_dic_sorted

strs_trees_percent_dic_2
strs_trees_percent_dic_sorted_2

strs_tree_counts

In [273]:
bad_ash

,FID,ID,UNIQUEID,ADDRESS,SUFFIX,STREET,ONSTR,FROMSTR,TOSTR,SIDE,...,GUARDSZ,EAB,INV_DATE,INV_TIME,INSPECT_DT,INSPECT_TM,NOTES,ACTIVE,COMMONNAME,GENUS
25,25,57063.0,JS 20150130090142,728,,10TH ST,10TH ST,BASELINE RD,CASCADE AV,Front,...,Unassigned,Unassigned,2015-01-30 00:00:00,,2015-07-10 00:00:00,11:40:29,,1.0,Green Ash,Fraxinus
132,132,49308.0,CC 20150108104820,1026,,10TH ST,10TH ST,PEARL ST,SPRUCE ST,Front,...,Unassigned,Unassigned,2015-01-08 00:00:00,,2015-09-02 00:00:00,03:28:26,"Crown defects, trunk wound",1.0,Green Ash,Fraxinus
199,199,49307.0,CC 20150108104651,2026,,10TH ST,10TH ST,PEARL ST,SPRUCE ST,Front,...,Unassigned,Group C,2015-01-08 00:00:00,,2016-12-29 00:00:00,12:27:04,,1.0,Green Ash,Fraxinus
213,213,49011.0,CC 20150105105102,2455,,10TH ST,10TH ST,PORTLAND PL,MAXWELL AV,Front,...,Unassigned,Group C,2015-01-05 00:00:00,,2016-12-29 00:00:00,12:27:04,,1.0,Green Ash,Fraxinus
217,217,45927.0,CC 20141219132638,2800,,10TH ST,10TH ST,BALSAM AV,CEDAR AV,Front,...,Unassigned,Unassigned,2014-12-19 00:00:00,,2015-07-10 00:00:00,11:40:29,growing into fence,1.0,Green Ash,Fraxinus
251,251,63302.0,JS 20150219074435,3004,,10TH ST,DELLWOOD AV,11TH ST,10TH ST,Right,...,Unassigned,Unassigned,2015-02-19 00:00:00,,2015-07-10 00:00:00,11:40:29,,1.0,Green Ash,Fraxinus
254,254,63308.0,JS 20150219074707,3004,,10TH ST,DELLWOOD AV,11TH ST,10TH ST,Right,...,Unassigned,Unassigned,2015-02-19 00:00:00,,2015-09-16 00:00:00,04:10:48,<,1.0,Green Ash,Fraxinus
255,255,63309.0,JS 20150219074805,3004,,10TH ST,DELLWOOD AV,11TH ST,10TH ST,Right,...,Unassigned,Unassigned,2015-02-19 00:00:00,,2015-07-10 00:00:00,11:40:29,,1.0,Green Ash,Fraxinus
316,316,63027.0,JS 20150217093646,3176,,10TH ST,10TH ST,EVERGREEN AV,FOREST AV,Front,...,Unassigned,Unassigned,2015-02-17 00:00:00,,2015-07-10 00:00:00,11:40:29,,1.0,Green Ash,Fraxinus
332,332,63428.0,JS 20150219131058,3233,,10TH ST,10TH ST,DEADEND,FOREST AV,Front,...,Unassigned,Unassigned,2015-02-19 00:00:00,,2015-07-10 00:00:00,11:40:29,stump sprouts,1.0,Green Ash,Fraxinus
